In [1]:
import pandas as pd
import requests
from urllib.parse import quote
import folium

In [21]:
df = pd.read_csv('./서울소방서.csv')

In [22]:
with open('./data1/kakao.txt') as file:
    kakao_key = file.read()

In [23]:
lat_list, lng_list = [], []
for i in df.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df.주소[i])}'
    result = requests.get(url,
        headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.관서명[i])

은평소방서-수색-119 안전센터


In [24]:
df[df.관서명.isin(["은평소방서-수색-119 안전센터"])].주소
# 여기에서 지우고 할것인지 따로 검색해서 넣어줄것인지 결정 할 수 있음

30    서울특별시 은평구 수색로 294 (수색동)
Name: 주소, dtype: object

#  방법1 :  주소를 카카오맵으로 검색해서 바꾸기

In [26]:
df.loc[30]

관서명           은평소방서-수색-119 안전센터
주소      서울특별시 은평구 수색로 294 (수색동)
전화번호                02-376-0119
Name: 30, dtype: object

In [28]:
# 카카오 맵으로 검색해서 수동으로 바꿔주기
df.loc[30]['주소'] = '서울 은평구 수색로 320'
df.loc[30]

관서명     은평소방서-수색-119 안전센터
주소         서울 은평구 수색로 320
전화번호          02-376-0119
Name: 30, dtype: object

In [ ]:
# 위도랑 경도를 하나씩 검색해서 넣어도 됨
url = f'{search_url}?query={quote("서울 은평구 수색로 320")}'
result = requests.get(url,
                      headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
print(float(result['documents'][0]['x']))
print(float(result['documents'][0]['y']))

In [ ]:
lat_list, lng_list = [], []
for i in df.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df.주소[i])}'
    result = requests.get(url,
        headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.관서명[i])

In [ ]:
df['위도'] = lat_list
df['경도'] = lng_list

# 2방법 : 지우고 수행하기

In [8]:
df.drop(30, axis=0, inplace=True)

In [9]:
lat_list, lng_list = [], []
for i in df.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df.주소[i])}'
    result = requests.get(url,
        headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df.관서명[i])

In [10]:
df['위도'] = lat_list
df['경도'] = lng_list

# 공통 : 지도 시각화

In [20]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=f'{df.관서명[i]}',
        icon=folium.Icon(icon ="glyphicon glyphicon-home")
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px"> 서울소재 소방서 위치 </h3>'
map.get_root().html.add_child(folium.Element(title_html))
map